In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, Settings, SQLDatabase, ServiceContext
from llama_index.core.node_parser import SentenceSplitter   
from llama_index.vector_stores.supabase import SupabaseVectorStore
import supabase
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from sqlalchemy import create_engine
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.text_splitter import TokenTextSplitter
from llama_index.core.node_parser import SimpleFileNodeParser


e:\MultiQuery RAG\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
GOOGLE_API_KEY="YOUR_GEMINI_API_KEY"
os.environ["GOOGLE_API_KEY"]= GOOGLE_API_KEY

In [3]:
import os
Settings.llm=Gemini()
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 5000
Settings.chunk_overlap = 400

In [4]:
SUPABASE_PROJECT_URL="YOUR_SUPABASE_PROJECT_URL/"
SUPABASE_API_KEY="SUPABASE_API_KEY"

In [5]:
supabase_= supabase.create_client(SUPABASE_PROJECT_URL, SUPABASE_API_KEY)
# res = supabase_.storage.create_bucket("documents")
# res=supabase_.storage.from_("documents").upload('PDFs/samplepdf.pdf','PDFs/samplepdf.pdf')

In [6]:
DB_CONNECTION="YOUR_POSTGRESQL_DB_URI"


In [7]:
db_engine=create_engine(DB_CONNECTION)
sql_db=SQLDatabase(db_engine)

In [8]:
sql_query_engine=NLSQLTableQueryEngine(
    sql_database=sql_db
)

In [9]:
documents = SimpleDirectoryReader("PDFs").load_data()
documents

[Document(id_='e1b27cb5-f883-4d16-93f2-5d43e2d96317', embedding=None, metadata={'page_label': '1', 'file_name': 'EG-37-100.pdf', 'file_path': 'e:\\MultiQuery RAG\\PDFs\\EG-37-100.pdf', 'file_type': 'application/pdf', 'file_size': 182503, 'creation_date': '2024-09-25', 'last_modified_date': '2024-08-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='1\nDate: \nAir-cooled Water-cooled Type:\n# of Stages:\n3* acfma,e\n4* Full Load Operating Pressure bpsigb\n5 Maximum Full Flow Operating Pressure cpsigc\n6Drive Motor Nominal Ratinghp\n7Drive Motor Nominal Efficiencypercent\n8Fan Motor Nominal Rating (if applicable)hp\n9Fan Motor Nominal Efficiency percent\n10* Total Package Input Power at Zero FlowekWe\n11 kWd\n12* kW/100 cfme\n13 Percent \n*For models th

In [10]:
splitter=SentenceSplitter()
vector_store = SupabaseVectorStore(
    postgres_connection_string=DB_CONNECTION,
    dimension=768,
    collection_name="sample_demo",
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents,storage_context=storage_context,embed_model=embed_model,transformations=[splitter])

In [11]:
from llama_index.core.query_engine import SQLAutoVectorQueryEngine, RetrieverQueryEngine
from llama_index.core.tools.query_engine import QueryEngineTool
from llama_index.core.indices.vector_store import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo

In [34]:
vector_store_info = VectorStoreInfo(
    content_info="Technical Documents about various Elgi compressors",
    metadata_info=[
        MetadataInfo(name="title", type="str", description="Technical Data of the Compressor"),
    ],
)
vector_auto_retriever= VectorIndexAutoRetriever(index=index,similarity_top_k=100, vector_store_info=vector_store_info)

retriever_query_engine = RetrieverQueryEngine(retriever=vector_auto_retriever)

In [35]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over a table containing: "
        "Customers, Gadgets and their pricing etc"
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    description=f"Technical Data about various Elgi Compressors",
)

In [36]:
query_engine = SQLAutoVectorQueryEngine(
    sql_tool, vector_tool
)


In [17]:

response = query_engine.query("What is the capacity of particulate filter?")
response.response

Querying other query engine: The question asks about 'particulate filter capacity', which is a technical specification. Choice 2 mentions technical data about Elgi Compressors, making it more relevant than choice 1, which focuses on natural language to SQL translation.


'The particulate filter has a capacity of 20 - 60 cfm. \n'

In [141]:

response = query_engine.query("suggest me some 15 Litre and 20 Litre PET Compressors",)
response.response

Querying other query engine: The query asks for specific compressor types (PET) and sizes (15 Litre and 20 Litre), suggesting a need for technical data about compressors, which is what choice 2 describes.


'The TS 15 L B PET and MS 20 L B PET models are available in 15 and 20 Litre capacities. \n'

In [142]:

response = query_engine.query("how much does a laptop cost",)
response.response

Querying SQL database: This choice mentions a table containing pricing information, which is relevant to the question about the cost of a laptop.
SQL query: SELECT price FROM products WHERE product_name LIKE 'Laptop%';
SQL response: A laptop costs $1000.00. 

Transformed query given SQL response: New question: None 

query engine response: This document contains technical specifications for various air compressors. 



'The SQL query and response tell us that a laptop costs $1000.00. The vector store query and response are not relevant to the original question. \n\nTherefore, the answer to the original question "how much does a laptop cost" is **$1000.00**. \n'

In [32]:

response = query_engine.query("What is the total cost")
response.response

Querying SQL database: The query 'What is the total cost' suggests a calculation involving pricing information, which is mentioned in summary 1.
SQL query: SELECT SUM(quantity * price) AS total_cost FROM orders o JOIN products p ON o.product_id = p.product_id
SQL response: The total cost is $3050.00. 

Transformed query given SQL response: New question: None 

query engine response: This document contains information about ELGi air compressors. 



'The total cost of all orders is $3050.00. \n'

In [33]:

response = query_engine.query("What is the total cost product wise")
response.response

Querying SQL database: Choice 1 mentions a table containing Gadgets and their pricing, which is relevant to calculating total cost product-wise.
SQL query: SELECT p.product_name, SUM(o.quantity * p.price) AS total_cost FROM products p JOIN orders o ON p.product_id = o.product_id GROUP BY p.product_name
SQL response: The total cost for each product is:

* **Smartphone:** $600.00
* **Laptop:** $2000.00
* **Headphones:** $450.00 

Transformed query given SQL response: New question: **None** 

query engine response: This question cannot be answered from the given context. 



'The total cost for each product is:\n\n* **Smartphone:** $600.00\n* **Laptop:** $2000.00\n* **Headphones:** $450.00 \n'

In [48]:

response = query_engine.query("how much would it cost for 5000 smartphones")
response.response

Querying SQL database: This choice mentions a table containing Gadgets and their pricing, which is relevant to determining the cost of smartphones.
SQL query: SELECT price * 5000 FROM products WHERE product_name LIKE 'Smartphone';
SQL response: The cost for 5000 smartphones would be $3,000,000. 

Transformed query given SQL response: None 

query engine response: This is a question answering system. Please provide a question. 



'The cost for 5000 smartphones would be $3,000,000. \n'

In [67]:
response.metadata

{'09ee80c4-4eb7-426e-a38e-fe1fa6fb1417': {'sql_query': 'SELECT p.product_name, SUM(o.quantity * p.price) AS total_sales FROM products p JOIN orders o ON p.product_id = o.product_id GROUP BY p.product_name ORDER BY total_sales DESC',
  'result': [('Laptop', Decimal('2000.00')),
   ('Smartphone', Decimal('600.00')),
   ('Headphones', Decimal('450.00'))],
  'col_keys': ['product_name', 'total_sales']},
 'sql_query': 'SELECT p.product_name, SUM(o.quantity * p.price) AS total_sales FROM products p JOIN orders o ON p.product_id = o.product_id GROUP BY p.product_name ORDER BY total_sales DESC',
 'result': [('Laptop', Decimal('2000.00')),
  ('Smartphone', Decimal('600.00')),
  ('Headphones', Decimal('450.00'))],
 'col_keys': ['product_name', 'total_sales'],
 '6de094a1-8b55-47e8-8d67-ece951a3103e': {'page_label': '8',
  'file_name': 'eg-series-50hz.pdf',
  'file_path': 'e:\\MultiQuery RAG\\PDFs\\eg-series-50hz.pdf',
  'file_type': 'application/pdf',
  'file_size': 2149242,
  'creation_date': '2